<div class="alert alert-block alert-success">
<p style="font-size:20px"><b>Step 1: Data Preparation</b></p> 
Load data collected from DL_Cash project to MongoDb database.  This is the raw data set without any further preprocessing.
Preprocessing is done in the next few steps.
</div>    

In [1]:
import sys

oneDrive_root={}
oneDrive_root[1]="C:\\Users\\Chihyang\OneDrive for Business\\"
oneDrive_root[2]="C:\\Users\\Chihyang\OneDrive for Business\\"
oneDrive_root[3]="C:\\Users\\tsaic\\OneDrive - State University of New York at New Paltz\\"  # laptop
WNF_lib = ".//WNF_lib//"

site=3   # the short or long business OneDrive directory name
lib_dir=oneDrive_root[site]+'\\Prudentia\\Tsaipy'
# append additional library path for this study
sys.path.append(lib_dir)
sys.path.append(WNF_lib)

In [2]:
import pandas as pd

from pymongo import MongoClient
import pymongo
from dbconnect import mongodb as mdb
import numpy as np
import pprint
from WFN_lib.cluster_classify import cluster_analysis

### Input parameters

In [3]:
db_name = "Windfarm_6000"
collection_structure = "varStructures"
collection_data = "Data_S6000"
collection_result = "Results_S6000"

infile_name1 = ".//Results//data_sets//Complete_Features_Rating_Table.xlsx"
infile_name2 = ".//Results//data_sets//MinMax_data.xlsx"


<div class="alert alert-block alert-success">
<p style="font-size:20px"><b>Connect to the database:</b></p> 
<ul>
    <li>client: equivalent to <b>conn</b> in the return of a SQL connection request.</li>
    <li>Check if the database with name <b>db_name</b> exists.  If yes, drop it to re-create a new one.</li>
    <li>db: The reference to the database defined in "<b>db_name</b>"</li>
    <li>The database is not actually created until data are put into it.</li>
</ul>
</div>        

In [4]:
## Create a new database, db_name, in MongoDB
## If db_name exists, delete it first and then create a new one

client = MongoClient('localhost', 27017)  # connect to the db engine
dbnames = client.list_database_names()  # find all existing databases
print("Existing databases before adding the new one: ",dbnames)
# delete the database, db_name, if exists
if db_name in dbnames:   
    client.drop_database(db_name)
else:
    print(f"Database, {db_name}, does not exist!!!")

# create a new database, db_name
print(f"db_name: {db_name}")
db = client[db_name]  # create a new database  

## Check if the database is created.  At this point, it is not found since no data created in it yet.
dbnames = client.list_database_names()  # find all existing databases
print("Databases after adding the new one: ", dbnames)
if db_name in dbnames:   # check if db_name is created
    print(f"Database, {db_name}, successfully created!!")
else:
    print(f"Database, {db_name}, not found!!")

Existing databases before adding the new one:  ['Windfarm_6000', 'admin', 'config', 'fin_clustering2002_2019', 'fin_clustering2005_2022', 'fin_clustering2019', 'local']
db_name: Windfarm_6000
Databases after adding the new one:  ['admin', 'config', 'fin_clustering2002_2019', 'fin_clustering2005_2022', 'fin_clustering2019', 'local']
Database, Windfarm_6000, not found!!


<div class="alert alert-block alert-info">
<p style="font-size:20px"><b>Create a database collection (table) for the "structures" of various collections in this step:</b></p> 
<ul>
    <li><b>coll_struc</b>: The reference to the collection (equivalent to a SQL table) "<b>collection_structure</b>"</li>
    <li>Note that a collection is not actually created until at least a document (record) is inserted to the collection.</li>
    <li>Each row of this table store the field structure of a table.  Thus, the number of rows in this table equals the number of tables (collections) in the database</li>
</ul>
</div> 

In [5]:
## Drop the collection, collection_structure and recreate it anew.
try:
    db.validate_collection(collection_structure)  # Try to validate a collection   
    db[collection_structure].drop()
    print(f"Collection (table), {collection_structure}, dropped")
except pymongo.errors.OperationFailure:  # If the collection doesn't exist
        print(f"Collection, {collection_structure}, doesn't exist") 
     
####### Create a new collections ##############################        
coll_struc=db[collection_structure]   # collection (table) name

####### Adding documents (rows) to the collection (table) ##########
coll_struc.insert_one({"_id":"V31_PLUS","category":"extracted_data","file_name":"Complete_Features_Rating_Table.xlsx","directory":".//Results//data_sets//",
                           "type":"31 feature variables","in_collection":"Data_S6000",
                       "desciption":"the 31 featured variables, 1-5 rating, and Random Forest classification probability (0,1) from Nguyen et al. (2021)"})
coll_struc.insert_one({"_id":"V31_MinMax", "category":"scaled_data","file_name":"MinMax_data.xlsx","directory":".//Results//data_sets//",
                           "type":"31 scaled feature variables","in_collection":"Data_S6000",
                       "desciption":"the 31 featured variables in V31_PLUS scaled to [0,1]"})
coll_struc.insert_one({"_id":"S6000_wav", "category":"source data","file_name":"Hn_nnnn.wav","directory":".//wav//wav_files//",
                           "type":"8192 rate", "in_collection":"None","desciption":"The 6000 10 s. wav files with 8192 rate"})
coll_struc.insert_one({"_id":"IOA_data", "category":"filtered_data","file_name":"Hn_nnnn_bandFrq.npy","directory":".//Windfarm_IOA//IOA_NPY_data_S6000_819//",
                           "type":"819 rate", "in_collection":"Data_S6000", "description":"The 6000 10 s. octave band filtered and A-weighted data"})
#### Create an additional index on "version" 
####### This is in addition to the default/irremovable index _id.
#result = coll_struc.create_index([('id', pymongo.ASCENDING)],unique=True)
sorted(list(coll_struc.index_information()))  # show the indices        

Collection, varStructures, doesn't exist


['_id_']

#### Test access to the newly created collection

In [6]:
dbnames = client.list_database_names()  # find all existing databases
print(dbnames)
## Test retrieval of fields
res = coll_struc.find_one({"category": "extracted_data"})
# show elements in res
for x in res:
    ## get the content of the field, x
    print(f"res[{x}]: {res[x]}")

['Windfarm_6000', 'admin', 'config', 'fin_clustering2002_2019', 'fin_clustering2005_2022', 'fin_clustering2019', 'local']
res[_id]: V31_PLUS
res[category]: extracted_data
res[file_name]: Complete_Features_Rating_Table.xlsx
res[directory]: .//Results//data_sets//
res[type]: 31 feature variables
res[in_collection]: Data_S6000
res[desciption]: the 31 featured variables, 1-5 rating, and Random Forest classification probability (0,1) from Nguyen et al. (2021)


<div class="alert alert-block alert-info">
<p style="font-size:20px"><b>Create a database collection (table) for the data:</b></p> There are three datasets in this data based on the same source wav files.  The Minmax scaler created to scale the data is saved under <b>.//trained_models</b>.
<ul>
    <li><b>v31</b>: The 31 feature variables per Nguyen</b>"</li>
    <li><b>v31_minmax</b>:The min_max scaled 31 variables into the range of (0,1).</li>
    <li><b>IOA</b>:The octave band filtered, A-weighted data</li>
</ul>
</div> 

#### Read the data source from an Excel file with three worksheets into pandas DataFrames

#### Insert rows to collection_data collection
- This process is done row-by-row using the insert_one() function
- Each row obtains its data from the three sources, two from the dataframes and one from reading the npy file.
- An additional index is created based on the stock __id__ field.

In [7]:
### drop the dataset collection, collection_data
try:
    db.validate_collection(collection_data)  # Try to validate a collection   
    db[collection_data].drop()
    print(f"Collection (table), {collection_data}, dropped")
except pymongo.errors.OperationFailure:  # If the collection doesn't exist
        print(f"Collection, {collection_data}, doesn't exist") 

coll_data=db[collection_data]   # collection (table) name 


Collection, Data_S6000, doesn't exist


In [8]:
### For the 31 feature  ########################################
## Retrieve the source data from the Excel file, infile_name1
df_v31=pd.read_excel(infile_name1,header=0,index_col=0)
print(df_v31.shape)
print(df_v31.head(3))

(6000, 35)
        Site filename    Rating  Prob_AM_RdmFrst  spectralCentroid  \
H1_0001   H1   s1.wav  1.091934         0.174275         63.081251   
H1_0002   H1   s2.wav  1.094161         0.019092         57.650009   
H1_0003   H1   s3.wav  4.967208         0.648424         75.049887   

         spectralCrest  spectralDecrease  spectralEntropy  spectralFlatness  \
H1_0001      28.079092        -92.466205         0.143241          0.010610   
H1_0002      30.680069       -259.422066         0.067829          0.011763   
H1_0003      25.424498        -22.843544         0.217764          0.008283   

         spectralFlux  ...          PR   Fo  AMfactor       DAM  \
H1_0001  1.087143e-07  ...   69.138538  0.9  0.239249  1.339453   
H1_0002  4.547078e-05  ...    7.843309  0.8  0.111174  1.217609   
H1_0003  3.236673e-06  ...  104.108914  0.7  0.827716  7.589734   

         peakloc_unweightedSPL       L63      L125      L250      L500  \
H1_0001                    0.2  3.403435  2.6850

In [9]:
## read in the Minmax scaled data processed earlier for verification purpose
## We need scale the data to [0,1] again in order to obtain and save the scaler for later use.
df_v31minmax_veri=pd.read_excel(infile_name2,header=0,index_col=0)
print(df_v31minmax_veri.shape)
print(df_v31minmax_veri.head(3))

(6000, 31)
         spectralCentroid  spectralCrest  spectralDecrease  spectralEntropy  \
H1_0001          0.012519       0.871260          0.999324         0.142351   
H1_0002          0.008121       0.957792          0.998104         0.065464   
H1_0003          0.022211       0.782944          0.999833         0.218333   

         spectralFlatness  spectralFlux  spectralKurtosis  \
H1_0001          0.010966      0.001331          0.004294   
H1_0002          0.012185      0.557133          0.002559   
H1_0003          0.008507      0.039656          0.000540   

         spectralRolloffPoint  spectralSkewness  spectralSlope  ...        PR  \
H1_0001              0.021432          0.078659       0.997918  ...  0.064817   
H1_0002              0.007925          0.068858       0.622292  ...  0.007353   
H1_0003              0.040809          0.027030       0.967046  ...  0.097601   

               Fo  AMfactor       DAM  peakloc_unweightedSPL       L63  \
H1_0001  1.000000  0.047060 

In [10]:
## Minmax scaling the 31 feasure variables: columns of df_v31[4:35]
## The first 4 columns are not scaled. 
## Check if the first three rows of the scaled data are the same as those from df_v31minmax_veri
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_v31minmax = scaler.fit_transform(df_v31.iloc[:,4:35])
df_v31minmax = pd.DataFrame(df_v31minmax, columns=df_v31.columns[4:35], index=df_v31.index)
print(df_v31minmax.shape)
## Check the first three rows of the scaled data with those from df_v31minmax_veri
print(df_v31minmax.head(3))
## Save the scaler to be used for scaling new data
import joblib
joblib.dump(scaler, './/trained_models//Minmax_scaler_v31_S6000.joblib')


(6000, 31)
         spectralCentroid  spectralCrest  spectralDecrease  spectralEntropy  \
H1_0001          0.012519       0.871260          0.999324         0.142351   
H1_0002          0.008121       0.957792          0.998104         0.065464   
H1_0003          0.022211       0.782944          0.999833         0.218333   

         spectralFlatness  spectralFlux  spectralKurtosis  \
H1_0001          0.010966      0.001331          0.004294   
H1_0002          0.012185      0.557133          0.002559   
H1_0003          0.008507      0.039656          0.000540   

         spectralRolloffPoint  spectralSkewness  spectralSlope  ...        PR  \
H1_0001              0.021432          0.078659       0.997918  ...  0.064817   
H1_0002              0.007925          0.068858       0.622292  ...  0.007353   
H1_0003              0.040809          0.027030       0.967046  ...  0.097601   

               Fo  AMfactor       DAM  peakloc_unweightedSPL       L63  \
H1_0001  1.000000  0.047060 

['.//trained_models//Minmax_scaler_v31_S6000.joblib']

## Store data in the collection Data_S6000
- First three columns: _id, site, file_name
- v31: 31 columns for the original 31 feature variables
- v31_minmax: minmax scaled v31 variables
- IOA: four bands 25-100Hz,50-200Hz, 100-400Hz, 200-800Hz, each with 100 values (No threshold, 819 Hz frame version), 100 values for each sample in each band.

In [11]:
### For the 31 feature variables and the 31 scaled feature variables ########################################
v31_keys=list(df_v31minmax.columns)   # variable names (or column names in the collection), v31 and v31_minmax share the same keys
### For reading and organizing IOA data ###################################################################
dir_npy='.//Windfarm_IOA//IOA_NPY_data_S6000_819//'  # A-weighted 819 frame rate data
bands = ['025_100Hz','050_200Hz', '100_400Hz', '200_800Hz']
post_fix=['_025_100Hz.npy','_050_200Hz.npy', '_100_400Hz.npy', '_200_800Hz.npy']
IOA_keys = ['v'+str(i).zfill(3) for i in range(100)]   # for the 100 values extracted from 819 rate time series data
#############################################################################################################
#df_v31.head(3)
for i, idx in enumerate(df_v31.index):
    pymongo_data_dict=dict()
    row = df_v31.loc[idx]    # v31
    row_minmax = df_v31minmax.loc[idx]   #v31_minmax
    
    #print(f"{i}: {idx}")

    ### file information
    pymongo_data_dict["_id"]=idx
    pymongo_data_dict["site"]=row["Site"]
    pymongo_data_dict["file_name"]=row["filename"]

    pymongo_data_dict["v31"]=dict(); 
    pymongo_data_dict["v31_minmax"]=dict()
    pymongo_data_dict["IOA"]=dict()

    for key in v31_keys:  #  collect the values of the V31 and V31_minmax variables
        pymongo_data_dict["v31"][key]=row[key]          
        pymongo_data_dict["v31_minmax"][key]=row_minmax[key]
        
    ## for IOA data, collect the values of the 4 bands for each file
    for (band, pos) in zip(bands, post_fix):
        pymongo_data_dict['IOA'][band]=list(np.load(dir_npy+idx+pos))


    ## insert the document into the collection
    coll_data.insert_one(pymongo_data_dict)
    #if i>3: break

<div class="alert alert-block alert-info"><b>Create a collection for analysis result</b>
<ol>
<li>scores: The rating and Random forest result from Nguyen 2021 based on 31 summary variables.</li>
<li>clusters: clustering results based the 31 variables, K=3, K=4, K=5, K=6</li>
<li>freqency bands: from IOA rating (AMWG)
<li>classes, probs: the top five classes and corresponding probabilities from YAMNET
</ol>
</div>

### Create Result database content
In the update statement, the last 2 True/False fields specifies the upsert and multi flags.

- Upsert flag: If set to true, creates a new document when no document matches the query criteria.

- Multi flag: If set to true, updates multiple documents that meet the query criteria. If set to false, updates one document.

In [12]:
### drop the result set collection, collection_result
try:
    db.validate_collection(collection_result)  # Try to validate a collection   
    db[collection_result].drop()
    print(f"Collection (table), {collection_result}, dropped")
except pymongo.errors.OperationFailure:  # If the collection doesn't exist
        print(f"Collection, {collection_result}, doesn't exist") 

coll_result=db[collection_result]   # collection (table) name

Collection, Results_S6000, doesn't exist


In [13]:
# Result 1: From the nguyen's paper
## These two scores are also in the collection, collection_data (Data_S6000)
df_v31.head(3)  
scores = ["Rating","Prob_AM_RdmFrst"]
for i, idx in enumerate(df_v31.index):
    pymongo_result_dict=dict()
    #print(f"{i}: {idx}")
    row = df_v31.loc[idx]

    pymongo_result_dict["_id"]=idx
    pymongo_result_dict["site"]=row["Site"]
    #pymongo_result_dict["file_name"]=row["filename"]

    pymongo_result_dict["scores"]=dict()
    ###############################
    for s in scores:
        pymongo_result_dict["scores"][s]=row[s]
    ## insert the document into the collection
    coll_result.insert_one(pymongo_result_dict)
    #if i>3: break

In [14]:
## Result 2: Clustering result by Kmeans with K=2, 3, .., 10 and save the clustering models
### Cluster the 6000 samples into 2-10 clusters
df_S6000_cluster, _ =cluster_analysis(df_v31minmax, [2,3,4,5,6,7,8,9,10], random_state=42, scaler_type=None,n_init=10, max_iter=300, tol=0.0001,
                                 save_model_dir='.//trained_models//', model_file_name='Kmeans_S6000_v31_MinMax_K=')
print(df_S6000_cluster.head(3))
print(df_S6000_cluster.shape)

## Double check with the saved clustering result
infile_name3 = ".//Results//cluster_allotment//Cluster_S6000_MinMax_result.xlsx"
df_cluster=pd.read_excel(infile_name3,header=0,index_col=0)
print(df_cluster.head(3))
print(df_cluster.shape)

         K=2  K=3  K=4  K=5  K=6  K=7  K=8  K=9  K=10
H1_0001    0    2    3    0    5    6    0    0     3
H1_0002    1    0    1    4    2    5    3    3     7
H1_0003    0    1    3    0    5    6    0    1     2
(6000, 9)
         K=2  K=3  K=4  K=5  K=6  K=7  K=8  K=9  K=10
H1_0001    0    2    3    0    5    6    0    0     3
H1_0002    1    0    1    4    2    5    3    3     7
H1_0003    0    1    3    0    5    6    0    1     2
(6000, 9)


In [15]:
## Result 2: Clustering result
infile_name3 = ".//Results//cluster_allotment//Cluster_S6000_MinMax_result.xlsx"
df_cluster=pd.read_excel(infile_name3,header=0,index_col=0)
df_cluster.head(3)

pymongo_cluster_dict=dict()
for i, idx in enumerate(df_S6000_cluster.index):
    pymongo_cluster_dict[idx]=dict()
    #print(f"{i}: {idx}")
    row = df_cluster.loc[idx]

    ## mongodb does not allow np.int64 as a value, chagne it to int
    pymongo_cluster_dict[idx]["K=2"]=int(row["K=2"])
    pymongo_cluster_dict[idx]["K=3"]=int(row["K=3"])
    pymongo_cluster_dict[idx]["K=4"]=int(row["K=4"])
    pymongo_cluster_dict[idx]["K=5"]=int(row["K=5"])
    pymongo_cluster_dict[idx]["K=6"]=int(row["K=6"])
    pymongo_cluster_dict[idx]["K=7"]=int(row["K=7"])
    pymongo_cluster_dict[idx]["K=8"]=int(row["K=8"])
    pymongo_cluster_dict[idx]["K=9"]=int(row["K=9"])
    pymongo_cluster_dict[idx]["K=10"]=int(row["K=10"])
    

    ## Append the new result for idx to the document 
    #print(pymongo_cluster_dict[idx])
    coll_result.update_one({"_id":idx },{'$set' : {"clusters": pymongo_cluster_dict[idx]}}, False, False)
    #if i>3: break



In [16]:
## Result 3: from IOA results
infile_name4=".//Windfarm_IOA//IOA_Result_819_S6000//All6000_IOA_AW_819_NoThreshold.xlsx"
xls = pd.ExcelFile(infile_name4)
sheet_names = xls.sheet_names  # see all sheet names
print(sheet_names)
keys=None

### Get the data from each sheet
df_bands = dict()
for sht in sheet_names[1:]:
    df_bands[sht]=pd.read_excel(infile_name4,sheet_name=sht,header=0,index_col=0)
    
    #print(df_bands[sht].head(3))
    if sht==sheet_names[1]:
        indices = df_bands[sht].index  # get the index names of the first sheet, since they are the same for all sheets
    
keys = ['prominence','L5-L95']   # get the column names of the two IOA values to extract

############################################################


for i, idx in enumerate(indices):
    pymongo_sht_dict=dict()
    for sht in sheet_names[1:]:
        pymongo_sht_dict[sht]=dict()
        ## find the index of the row in coll_result, with the same id
        row = df_bands[sht].loc[idx]
        #print(f"sht = {sht}, idx = {idx}")

        for key in keys:  # skip the first first column, 'band', which is the parent key
            pymongo_sht_dict[sht][key]=row[key]
        ## append the document to the end of the document with the same id if the document with the same id exists, otherwise show error message

    #print(pymongo_sht_dict)
    coll_result.update_one({"_id":idx },{'$set' : {"IOA_rating":pymongo_sht_dict}}, False, False)

    #if i>3: break

        

['All', '025_100Hz', '050_200Hz', '100_400Hz', '200_800Hz']


In [17]:
## Result 4: YAMNET results
infile_name5=".//Results//classification//YAMNET_cls_results.xlsx"
df_H1=pd.read_excel(infile_name5,sheet_name="H1_cls",header=0,index_col=0)
df_H2=pd.read_excel(infile_name5,sheet_name="H2_cls",header=0,index_col=0)
### concatenate the two dataframes
df_H=pd.concat([df_H1,df_H2],axis=0)
classes=df_H.columns[:5]  # the first 5 columns are the class names
probs=df_H.columns[5:]  # the rest of the columns are the probabilities
print(classes, probs)
#############################################################################
#pymongo_class_dict=dict()
#pymongo_prob_dict=dict()

for i, idx in enumerate(df_H.index):
    pymongo_YAMNET_dict=dict()
    pymongo_YAMNET_dict['class']=dict()
    pymongo_YAMNET_dict['prob']=dict()
    ## find the index of the row in coll_result, with the same id
    #pymongo_YAMNET_dict['class'][idx]=dict()
    #pymongo_YAMNET_dict['prob'][idx]=dict()
    row = df_H.loc[idx]

    for s in classes:
        pymongo_YAMNET_dict['class'][s]=row[s]
    for s in probs:
        pymongo_YAMNET_dict['prob'][s]=row[s]

    #print(pymongo_class_dict[idx])
    #print(pymongo_prob_dict[idx])
    ## append the document to the end of the document with the same id if the document with the same id exists, 
    #coll_result.update({"id":idx },{'$set' : {"classes": pymongo_class_dict[idx], "probs":pymongo_prob_dict[idx]}}, True, True)
    coll_result.update({"_id":idx },{'$set' : {"YAMnet": pymongo_YAMNET_dict}}, True, True)



Index(['top_1', 'top_2', 'top_3', 'top_4', 'top_5'], dtype='object') Index(['prob_1', 'prob_2', 'prob_3', 'prob_4', 'prob_5'], dtype='object')


C:\Users\tsaic\AppData\Local\Temp\ipykernel_11748\4178203803.py:32: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  coll_result.update({"_id":idx },{'$set' : {"YAMnet": pymongo_YAMNET_dict}}, True, True)


In [18]:
## find the index column of the collection, collection_data
print(f"Index for collection {collection_structure}: {coll_struc.index_information()}")
print(f"Index for collection {collection_data}: {coll_data.index_information()}")
print(f"Index for collection {coll_result}: {coll_result.index_information()}")
## To create an additional index, use the following command
#result = coll_data.create_index([('column_name', pymongo.ASCENDING)],unique=True)
#where column_name is the name of the column to be indexed


Index for collection varStructures: {'_id_': {'v': 2, 'key': [('_id', 1)]}}
Index for collection Data_S6000: {'_id_': {'v': 2, 'key': [('_id', 1)]}}
Index for collection Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Windfarm_6000'), 'Results_S6000'): {'_id_': {'v': 2, 'key': [('_id', 1)]}}


<div class="alert alert-block alert-success">
<p style="font-size:20px"><b>What have you achieved:</b></p> 
At this point, we have created the following.
<ul>
    <li>Collection <b>varStructures</b>: a collection that lists the source file category, name, location etc. with a brief descriptionile.</li>
    <li>Collection <b>Data_S6000</b>: Source data pertaining to each of the 6000 files,  including the 31 feature variables, their min_max scaled version, and IOA data.</li>
    <li>Collection <b>Results_S6000</b>: Analysis results pertaining to each of the 6000 files,  including (1) the original two scoring variables, rating and random forest probability from Nguyen, 2021, (2) clustering analysis result for K=3,4,5,6, (3) IOA Rating, (4) YAMNET results (five first pick categories and their associated probabilities.</li>
</ul>
</div>   